In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [2]:
# Directorio donde vamos a guardar las entradas o imputs 
Data_Dir = "Data"
# Directorio donde vamos a tomar las mediciones
Measure_Dir = "mic_behringher_ecm8000"

